In [32]:
#패키지 임포트
import json
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
import urllib.parse
import requests

In [40]:
#Chromedriver path
chromedriver_path = 'chromedriver'

#크롬 드라이버 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')

# 크롬 드라이버 실행
driver = webdriver.Chrome(chromedriver_path, options=options)

# 데이터를 저장할 빈 데이터프레임 생성
urdata = pd.DataFrame(columns=['svname','urname', 'rkpo'])

# 페이지 수 범위 설정
page_start = 1
page_end = 3

# 원하는 페이지 로드 후 데이터 추출
for page_num in range(page_start, page_end+1):
    # 페이지 로드
    url = f'https://dundam.xyz/damage_ranking?page={page_num}&type=9&job=%E7%9C%9E%20%EC%9B%A8%ED%8E%80%EB%A7%88%EC%8A%A4%ED%84%B0&baseJob=%EA%B7%80%EA%B2%80%EC%82%AC(%EB%82%A8)&weaponType=%EC%A0%84%EC%B2%B4&weaponDetail=%EC%A0%84%EC%B2%B4&banList='
    driver.get(url)
    sleep(2)

    # 페이지 소스코드 가져오기
    html = driver.page_source

    # 데이터 추출
    soup = BeautifulSoup(html, 'html.parser')
    svname = soup.select('span.svname')
    urname = soup.select('span.urname')
    rkpo = soup.select('div.rkpo')
    for i, name in enumerate(urname):
        urname_plus = pd.DataFrame({'svname' : [svname[i].text],
                                    'urname': [name.text],
                                    'rkpo': [rkpo[i].select_one('span').text]})
        urdata = pd.concat([urdata,urname_plus], ignore_index = True)

    # 현재까지 진행된 페이지 수 출력
    print(f'page {page_num} complete')

# 크롬 드라이버 종료
driver.quit()

# 서버명 영문변환
urdata['svname'] = urdata['svname'].replace({'안톤' : 'anton',
                                            '카인': 'cain', 
                                            '힐더': 'hilder', 
                                            '시로코': 'siroco', 
                                            '프레이': 'prey', 
                                            '디레지에': 'diregie',
                                            '카시야스': 'casillas',
                                            '바칼' : 'bakal'})


urdata.to_csv(f'page{page_start}~{page_end} urdata.csv')

# 결과 확인
urdata

page 1 complete
page 2 complete
page 3 complete


,svname,urname,rkpo
0,cain,빵문검귀이,"49,897"
1,siroco,쿰,"49,627"
2,cain,し㏄,"49,551"
3,cain,새지타,"49,549"
4,hilder,닉넴이읍어,"49,543"
5,cain,석양공주,"49,511"
6,diregie,쿠노이치,"49,511"
7,prey,븝포칼립스,"49,506"
8,cain,안녕,"49,500"
9,diregie,노상훈,"49,486"


In [41]:
urdata = pd.load_csv('page1~3 urdata.csv')
chid = pd.DataFrame(columns=['characterId'])

for i in range(0 , len(urdata)) :
    svname = urdata.iloc[i, 0]
    urname = urdata.iloc[i, 1]
    urname_url = urllib.parse.quote(urname)
    #아이디 검색 API(URL 변환 필요)
    requestData = requests.get(f'https://api.neople.co.kr/df/servers/{svname}/characters?characterName={urname_url}&apikey=M23RMaIB2JlKPyRWYRBKDb6UtE3NDTrS')
    if requestData.status_code == 200 :
        jsonData = requestData.json()
    else :
        print(requestData.status_code, 'Request Bad')
        print(requestData.json())
        break
    jsonData = requestData.json()
    api_result = pd.DataFrame(jsonData['rows'])
    characterId_filter = pd.DataFrame({'characterId' : api_result.characterId})
    chid = pd.concat([chid, characterId_filter])


In [45]:
chid2 = chid.iloc[0].to_list()
chid2

['bf6b24c203111c6ede8c2faf4be2b9d6']

In [16]:
# 데이터 추출
soup = BeautifulSoup(html, 'html.parser')
svname = soup.select('span.svname')
urname = soup.select('span.urname')
rkpo = soup.select('div.rkpo')
for i, name in enumerate(urname):
    urname_plus = pd.DataFrame({'svname' : [svname[i].text],
                                'urname': [name.text],
                                'rkpo': [rkpo[i].select_one('span').text]})
    urdata = pd.concat([urdata,urname_plus], ignore_index = True)
   
urdata

,svname,urname,rkpo
0,카인,모든속성저항,"49,465"
1,카인,NT-1,"49,465"
2,디레지에,있찌,"49,465"
3,힐더,o페이탈o,"49,463"
4,카인,Rm9yZ290dGVu,"49,460"
5,카인,테뷰,"49,451"
6,카인,최고의무대,"49,448"
7,힐더,민트크림소스,"49,443"


In [17]:

urdata

0       Cain
1       Cain
2    Delezie
3     Hilder
4       Cain
5       Cain
6       Cain
7     Hilder
Name: svname, dtype: object

In [15]:
urdata = pd.DataFrame(columns=['svname','urname', 'rkpo'])